In [1]:
from auxiliary import loadData_staticTargetAddrMatch, prepare_data_loaders
from algorithms import RssPosAlgo_NeuralNet_MLP4layer
from algorithms import RssPosAlgo_NeuralNet_supervisedTrainingLoop
from algorithms import RssPosAlgo_NearestNeighbour_Interpolation
from algorithms import RssPosAlgo_NearestNeighbour_GetKmeansDb
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

### prepare model, dataloaders and training parameters

In [2]:
datajsonpath = "../experiments/exp002_20240907_sna_kadirburak/data-tshark/data_nosnap.json"
inp_rss_vals, gt_locations = loadData_staticTargetAddrMatch(datajsonpath, second_hold = 5, shuffle=False, 
                                                            target_addresses=["4c:77:6d:92:b3:60", 
                                                                              "4c:77:6d:5f:ea:e0", 
                                                                              "4c:77:6d:a0:45:c0"], snap250ms=False)
epochs           = 601
batch_size       = 8
train_test_split = 0.5

train_loader, test_loader, xtr, ytr, xts, yts = prepare_data_loaders(inp_rss_vals, gt_locations, 
                                                                     batch_size = 32, 
                                                                     train_test_split = train_test_split)

print("Subset sizes | train:", xtr.shape[0], ", test:",xts.shape[0])
MLP = RssPosAlgo_NeuralNet_MLP4layer()
MLP.train()

MLP_criterion = nn.MSELoss(reduction='mean')
MLP_optimizer = optim.Adam(MLP.parameters(), lr=3e-4)

Subset sizes | train: 181 , test: 181


### train model

In [3]:
MLP = RssPosAlgo_NeuralNet_supervisedTrainingLoop(train_loader = train_loader, 
                                                         test_loader  = test_loader,
                                                         model        = MLP, 
                                                         criterion    = MLP_criterion, 
                                                         optimizer    = MLP_optimizer, 
                                                         epochs       = epochs,
                                                         testfreq     = 20) # testfreq is in epochs

Epoch [1/601] test loss: 6.969, training loss: -1.000
Epoch [21/601] test loss: 1.352, training loss: 0.960
Epoch [41/601] test loss: 1.317, training loss: 0.949
Epoch [61/601] test loss: 1.282, training loss: 0.939
Epoch [81/601] test loss: 1.271, training loss: 0.932
Epoch [101/601] test loss: 1.260, training loss: 0.926
Epoch [121/601] test loss: 1.252, training loss: 0.921
Epoch [141/601] test loss: 1.243, training loss: 0.915
Epoch [161/601] test loss: 1.233, training loss: 0.910
Epoch [181/601] test loss: 1.224, training loss: 0.905
Epoch [201/601] test loss: 1.215, training loss: 0.900
Epoch [221/601] test loss: 1.208, training loss: 0.895
Epoch [241/601] test loss: 1.198, training loss: 0.890
Epoch [261/601] test loss: 1.191, training loss: 0.885
Epoch [281/601] test loss: 1.189, training loss: 0.880
Epoch [301/601] test loss: 1.182, training loss: 0.875
Epoch [321/601] test loss: 1.173, training loss: 0.870
Epoch [341/601] test loss: 1.165, training loss: 0.865
Epoch [361/601]

### save the model

In [4]:
torch.save(MLP.state_dict(), 'savedmodels/dev006_MLP4layer.pth')

### evaluate model

In [5]:
MLP = RssPosAlgo_NeuralNet_MLP4layer()
MLP.load_state_dict(torch.load('savedmodels/dev006_MLP4layer.pth'));
MLP.eval();

In [6]:
db_kmeans = RssPosAlgo_NearestNeighbour_GetKmeansDb(xtr, ytr, num_clusters=5)
meanerror_nene_interp = 0;
meanerror_mlp         = 0;
for test_idx, x_test_sample in enumerate(xts): 
    loc_pred_mlp           = MLP(x_test_sample)
    loc_pred_nene_interp   = RssPosAlgo_NearestNeighbour_Interpolation(x_test_sample, db_kmeans)
    meanerror_mlp         += (yts[test_idx].numpy() - loc_pred_mlp.detach().numpy())**2
    meanerror_nene_interp += (yts[test_idx].numpy() - loc_pred_nene_interp)**2
print("MLP        :", np.linalg.norm(meanerror_mlp/(test_idx+1)))
print("NeNe+Interp:", np.linalg.norm(meanerror_nene_interp/(test_idx+1)))

MLP        : 1.6761531
NeNe+Interp: 1.983491520059886


### test a few samples

from test set

In [7]:
sampleid = 13
loc_pred_mlp           = MLP(xts[sampleid])
loc_pred_nene_interp   = RssPosAlgo_NearestNeighbour_Interpolation(xts[sampleid], db_kmeans)
meanerror_mlp         += (yts[sampleid].numpy() - loc_pred_mlp.detach().numpy())**2
meanerror_nene_interp += (yts[sampleid].numpy() - loc_pred_nene_interp)**2
print("Actual position  :", yts[sampleid].numpy())
print("MLP prediction   :", loc_pred_mlp.detach().numpy())
print("NeNe+Interp pred :", loc_pred_nene_interp)

Actual position  : [2.368339 0.867669]
MLP prediction   : [1.7114627 1.8513441]
NeNe+Interp pred : (2.0949473016797366, 2.2040911864975254)


---

from train set

In [8]:
sampleid = 20
loc_pred_mlp           = MLP(xtr[sampleid])
loc_pred_nene_interp   = RssPosAlgo_NearestNeighbour_Interpolation(xtr[sampleid], db_kmeans)
meanerror_mlp         += (ytr[sampleid].numpy() - loc_pred_mlp.detach().numpy())**2
meanerror_nene_interp += (ytr[sampleid].numpy() - loc_pred_nene_interp)**2
print("Actual position  :", ytr[sampleid].numpy())
print("MLP prediction   :", loc_pred_mlp.detach().numpy())
print("NeNe+Interp pred :", loc_pred_nene_interp)

Actual position  : [2.4450567 3.1718736]
MLP prediction   : [1.657197  2.9171662]
NeNe+Interp pred : (2.043017728595379, 3.5030880151907358)
